LIF neuron : $U[t+1]=\beta U[t] + WX[t+1]-R[t]$  
input synaptic : $I_{in}[t] = WX[t]$  
input spike : $X[t]$  
output spike : $S[t]$  
$\beta$ : both hyperparameter & learnable parameter

training networks : loss with weights  
$\frac{\partial L}{\partial W} = \frac{\partial L}{\partial S[t]} \frac{\partial S[t]}{\partial U[t]} \frac{\partial U[t]}{\partial I[t]} \frac{\partial I[t]}{\partial W}$  
$\frac{\partial S[t]}{\partial U[t]}=\{0, \infty\}$ => dead neuron probelm => surrogate function  
forward propagation은 같게 backpropagation=$\frac{\partial \tilde{S}}{\partial U} = \frac{1}{\pi} \frac{1}{(1+[U\pi]^2)}$

In [3]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import itertools

In [4]:
class LeakySurrogate(nn.Module):
    def __init__(self, beta, threshold=1.0):
        super(LeakySurrogate, self).__init__()

        self.beta = beta
        self.threshold = threshold
        self.spike_gradient = self.ATan.apply

    def forward(self, input_, mem):
        spk = self.spike_gradient(mem - self.threshold) # call heaviside
        reset = (self.beta*spk*self.threshold).detach()
        mem = self.beta*mem + input_ - reset
        return spk, mem
    
    @staticmethod  # 데코레이터 -> self 안씀
    class ATan(torch.autograd.Function):
        @staticmethod
        def forward(ctx, mem): # heaviside funtion
            spk = (mem > 0).float() # mem-threshold>0 -> return 1
            ctx.save_for_backward(mem) # forward 계산한 mem 저장
            return spk

        @staticmethod
        def backward(ctx, grad_output):
            (mem, )=ctx.saved_tensors
            grad = 1/ (1+(np.pi*mem).pow_(2))
            return grad

In [5]:
lif1 = LeakySurrogate(beta=0.9)

In [6]:
lif1 = snn.Leaky(beta=0.9) # 구현되어있다

사실 backpropagation은 모든 시간을 고려해야한다. 전의 loss의 sum과 지금의 loss를 더해야한다.

### setting decoding
rate coding : 정답인 label이 평균 firing이 가장 높게  
=> 정답의 U를 높게, 오답의 U는 낮게(weight 조절)  
** 결과는 spike로 판단하지만 계산과정에서 U(막전위)를 쓰겠다(연속값이라 계산 easy)

In [7]:
import torch

batch_size = 128
data_path='/tmp/data/mnist'

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [8]:
transform = transforms.Compose([
    transforms.Resize((28,28)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0,), (1,))
    ])



In [10]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

mnist_train = datasets.MNIST(root = './', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root = './', train=False, download=True, transform=transform)

--2026-02-10 04:38:33--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2026-02-10 04:38:34--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz.2’

MNIST.tar.gz.2          [    <=>             ]  33.20M   236KB/s    in 5m 27s  

2026-02-10 04:44:02 (104 KB/s) - ‘MNIST.tar.gz.2’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/

In [ ]:
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, drop_last=True)  

In [ ]:
# define network
num_inputs = 28*28
num_hidden = 1000
num_outputs = 10

num_steps = 25
beta = 0.95

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()

        spk2_rec = []
        mem2_rec = []
        
        for step in range(num_steps):
            cur1 = self.fc1(x)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

net = Net().to(device)

In [ ]:
# training
def print_batch_accuracy(data, targets, train=False):
    output, _ = net(data.view(batch_size, -1)) # output : spike 기록, _ : mem
    # _ : 알 필요 없는 정보를 이렇게 전달한다. 메모리 절약
    _, idx = output.sum(dim=0).max(1) # max(1) 한 행에서 가장 큰 클래스(값 아니고 인덱스) 찾아 저장
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f'Train set accuracy for a single minibatch: {acc*100:.2f}')

    else:
        print(f'Test set accuracy for a single minibatch: {acc*100:.2f}')

def train_printer(
        data, targets, epoch, counter, iter_counter, loss_hist, test_loss_hist, test_data, test_targets):
    print(f'Epoch {epoch}, Iteration {iter_counter}')
    print(f'Train Set Loss: {loss_hist[counter]:.2f}')
    print(f'Test Set Loss: {test_loss_hist[counter]:.2f}')
    print_batch_accuracy(data, targets, train=True)
    print_batch_accuracy(test_data, test_targets, train=False)
    print('\n')

In [ ]:
loss = nn.CrossEntropyLoss()

NameError: name 'nn' is not defined

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))